# Augment Single Example
This notebook shows you how to augment a single audio example using random pedalboards.

### Imports

In [8]:
from pedalboard import Compressor, PitchShift, HighpassFilter
import soundfile as sf
import librosa

import sys
sys.path.append("../code/")
from audio_augmentation import roll_pedal, pedal_dict, process_track

## Configure Pedal Chain
Here, we define an effect chain by specifying the effects we want to use in the correct order. Also, we assign a probability value to each pedal that states how likely the pedal is to be activated for each pedalboard roll. If you want all of your pedals to be applied every time, set all probabilities to 1.

In [1]:
pedal_config = [("compressor", .3),
                ("chorus", .3),
                ("reverb", .3),
                ("distortion", .3),
                ("lowpassfilter", .3),
                ("highpassfilter", .3),
                ("pitchshift", .3)
                ]

Next, we define a fallback effect chain. This one is needed in case none of the pedals pass the random roll, i.e. no effect is selected. In this case, the module will use this fallback effect chain as a default.

In [3]:
from pedalboard import Compressor, PitchShift, HighpassFilter

fallback = [HighpassFilter(cutoff_frequency_hz=300),
            PitchShift(semitones=1),
            Compressor(threshold_db=-10, ratio=1.3)]

## Roll Pedalboard
Here, we apply the ```roll_pedal``` function to each pedal, making use of the prebuilt parameter ranges in ```pedal_dict```.

In [26]:
pedal_rolls = [roll_pedal(pedal=pedal_dict[pedal]["pedal"],
                          param_dict=pedal_dict[pedal]["param_dict"],
                          pedal_prob=prob,
                          random_seed=None)
               for pedal, prob in pedal_config]

print(pedal_rolls)

[<pedalboard.Compressor threshold_db=-23.9524 ratio=1.91246 attack_ms=1 release_ms=100 at 000001DA12C4D590>, None, None, None, <pedalboard.Lowpass cutoff_frequency_hz=4832 at 000001DA928C9C00>, <pedalboard.Highpass cutoff_frequency_hz=387 at 000001DA928C86C0>, <pedalboard_native.PitchShift object at 0x000001DA950A9530>]


## Apply Pedal to Audio
Finally, we load an audio file and apply the random pedalboard using ```process_track```.

In [27]:
audio, sr = librosa.load("../audio_data/article_demo_tracks/base_example.mp3")
audio_processed = process_track(
  audio, sample_rate=sr,
  pedals=pedal_rolls, fallback=fallback
)

In [28]:
sf.write("../audio_data/article_demo_tracks/base_example_random.wav", audio_processed, sr)